# A synthetic DC/IP model
## Overview
Tina yesterday suggested a synthetic model that is kind of a mix inspired by both the Mjölkalunga (sediments) and Odarslöv (Dike). We have three main layers:
* 2 metres of dry sand
* 8 metres of saturated sand (aquifer)
* clay basement
plus two anomalies:
* a (silty?) block in the sand
* a near-vertical dyke

Assume a Terrameter LS2 measurement with 81 electrodes using the multiple-gradient array.

![Model](model.png)

In [ ]:
# We first import the needed modules
import numpy as np
import pygimli as pg
import pygimli.meshtools as mt  # submodule of pg
from pygimli.physics import ert  # subsubmodule

## The synthetic model
### The data scheme
We start with the data, creating a multi-gradient data with 81 electrodes, compute the geometric factors and display it. Note that the vertical axis does not refer to a pseudodepth but is a logical arrangement of data points so that every data is represented by a block of equal size, here of a Schlumberger (sl) type data. The data consist of four blocks with potential electrode distances of 1, 3, 5 and 7. One oblique line represents data with the same AB bipole.

## The model geometry
As next step, we build up the synthetic model starting with the function `createWorld` to create a box, into which layers are inserted. Next, we will create arectangular anomaly, and finally a dike, which will explain us the region markers a little bit.

We add the electrode as nodes to the geometry to ensure sufficient accuracy, and create a mesh of the model. Then we populate the model with resistivities by using a map.

## The forward modelling
We simply call the `simulate` function of the ERT module, passing the mesh, the data scheme, and the resistivity map. Additionally we provide a noise level for estimating an error and generating Gaussian noise that is added to the data.

## DC Inversion
We use the ERT manager class passing the data. The inversion is only another simple one-liner to which we can pass any options for mesh, regularization or inversion. We will play with the regularization parameter, the vertical weight, or we might use geostatistical regularization.